# 実行環境の作成

## Google Drive

Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Google Sheets

ライブラリのインストール

In [ ]:
!pip install --upgrade -q gspread

Google Sheetsの認証

In [ ]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## Google Cloud Storage

下記コードでGCPに接続

In [ ]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

2022/09/25 01:11:44.734989 Start gcsfuse/0.41.6 (Go version go1.18.4) for app "" using mount point: /content/statistics-hyogo
2022/09/25 01:11:44.750651 Opening GCS connection...
2022/09/25 01:11:46.337827 Mounting file system "statistics-hyogo"...
2022/09/25 01:11:46.338562 File system has been successfully mounted.


## ESTAT_APP_ID

In [ ]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

## pythonのライブラリ追加

In [ ]:
import urllib.parse
import urllib.request
import json
import pandas as pd

# 統計カード管理情報を取得・加工

## BigQueryから統計カード管理情報を読み込む

pandas-gbqのインストール

In [ ]:
!pip install pandas-gbq

BigQueryをDataFrameで取得

In [ ]:
def get_cards_management_dataframe():
  # プロジェクトの定義
  project_id='primal-buttress-342908'
  dataset_id = 'contents'
  table_id='cards'

  # クエリ
  query = f"""
  SELECT *
  FROM {dataset_id}.{table_id}
  """

  # dialect='standard' で標準SQLを使用
  df = pd.read_gbq(query, project_id, dialect='standard')

  return df

In [ ]:
df_cardmng = get_cards_management_dataframe()
df_cardmng

## 統計カードリストの作成

統計カード（cardId,cardTitle)のリストを作成する

In [ ]:
def get_cardliist(df_cardmng):
  #  'cardId'の指定があるデータを抽出
  df = df_cardmng.dropna(subset=['cardId'])
  
  # GroupBy
  df = df.groupby(['cardId','cardTitle','governmentType','statsDataId']).agg(
    {'categoryCode': lambda x: ','.join(x)
    })
  
  return df

In [ ]:
df_cardlist = get_cardliist(df_cardmng)
df_cardlist

,,,,categoryCode
cardId,cardTitle,governmentType,statsDataId,
Immigrant-prefecture,都道府県の入国者数,prefecture,10101,"A5201,A520101,A520102"
abandoned-cultivated-city,市区町村の耕地放棄面積,city,20203,C3109
abandoned-cultivated-prefecture,都道府県の耕地放棄面積,prefecture,10103,C3109
administrative-staff-city,市区町村の一般行政部門職員数,city,20204,D1202
administrative-staff-prefecture,都道府県の一般行政部門職員数,prefecture,10104,"D1201,D1202"
...,...,...,...,...
violence-prefecture,都道府県の暴力行為発生件数,prefecture,10105,E9501
waste-processed-prefecture,都道府県のごみ処理量,prefecture,10108,"H5612,H561201"
welfare-facilities-elderly-city,市区町村の老人福祉施設数,city,20210,"J2301,J2304"


# 統計カードのデータセット

## RESAS_API_KEY

In [ ]:
RESAS_API_KEY = '	02JYzLqUOfTNVfiTWjLlR2g0YwEQPQ7caYxcnZaZ'

地域コード一覧を取得する関数

In [ ]:
import json
import urllib.request

def generate_estat_areas(governmentType='prefecture',prefCode=0):

  # 都道府県の場合
  if governmentType == 'prefecture':
    pref_codes = [d.get('prefCode') for d in get_resas_preflist()]
    return [f'{n:02}'+'000' for n in pref_codes]
  
  # 市区町村の場合
  else:
    city_codes = [d.get('cityCode') for d in get_resas_citylist(prefCode)]
    return city_codes

北海道の市区町村数が１００以上あるので分割する

In [ ]:
# def generate_estat_city_areas():
#   prefCodes = [d.get('prefCode') for d in get_resas_preflist()]

#   res = []
#   for prefCode in prefCodes:
    
#     city_codes = [d.get('cityCode') for d in get_resas_citylist(prefCode)]

#     # 北海道の場合
#     if prefCode == 1 :
#       dict1 = {
#           'prefCode': prefCode ,
#           'areaCodes':city_codes[:99],
#       }
#       dict2 = {
#           'prefCode': prefCode ,
#           'areaCodes':city_codes[99:],
#       }
#       res.append(dict1)
#       res.append(dict2)

#     # 北海道以外の場合
#     else:
#       dict = {
#           'prefCode': prefCode ,
#           'areaCodes':city_codes,
#       }
#       res.append(dict)
  
#   return res

## 都道府県コードリスト

resas-APIから都道府県一覧を取得する

In [ ]:
"""
resas-APIから都道府県一覧を取得する関数
"""

def get_resas_preflist():
    url = 'https://opendata.resas-portal.go.jp/api/v1/prefectures'
    req = urllib.request.Request(url, headers={'X-API-KEY': RESAS_API_KEY})
    with urllib.request.urlopen(req) as response:
        res = json.loads(response.read().decode())
    return res['result']

estat-APIのリクエストパラメータ'cdArea'に都道府県コードをセットする

返却値はリスト

In [ ]:
"""
都道府県コード'28000'のリストを取得する関数
"""

def get_cdAreas_prefecture():

  res = []

  pref_codes = [d.get('prefCode') for d in get_resas_preflist()]
  pref_codes = [[str(c).zfill(5) for c in pref_codes]]
  
  dic = {'cdArea':",".join(generate_estat_areas())}
  res.append(dic)
  
  return res

In [ ]:
cdArea = get_cdAreas_prefecture()
print(cdArea)

## 市区町村コードリスト

resas-APIから市区町村一覧を取得する

In [ ]:
"""
resas-APIから市区町村一覧を取得する関数
"""

def get_resas_citylist(prefCode=0,designatedCity='all'):
  
    # 都道府県'prefCode'を指定
    # 0の場合は全都道府県の市区町村
    if prefCode == 0:
      url = 'https://opendata.resas-portal.go.jp/api/v1/cities'
    else:
      url = 'https://opendata.resas-portal.go.jp/api/v1/cities?prefCode={}'.format(prefCode)
    
    req = urllib.request.Request(url, headers={'X-API-KEY': RESAS_API_KEY})
    with urllib.request.urlopen(req) as response:
        res = json.loads(response.read().decode())


    # 政令指定都市　の指定　'join':政令指定都市統合　'split'：政令指定都市分割 'all'：全て
    if designatedCity == 'join':
      return list(filter(lambda x: x['bigCityFlag'] != '1', res['result']))
    elif designatedCity == 'split':
      return list(filter(lambda x: x['bigCityFlag'] != '2', res['result']))
    else:
      return res['result']

estat-APIのリクエストパラメータ'cdArea'に市区町村コードをセットする

返却値はリスト

In [ ]:
def get_cdAreas_city(prefCode=0):

  res=[]

  citylist = get_resas_citylist(prefCode=prefCode,designatedCity='all')
  city_codes = [d.get('cityCode') for d in citylist]

  # 市区町村が100以上ある場合
  if len(city_codes) >= 100 :
    for i in range(0, len(city_codes), 100):
      dic = {'cdArea':",".join(city_codes[i: i+100])}
      res.append(dic)

  # 市区町村が100未満の場合
  else:
    dic = {'cdArea':",".join(city_codes)}
    res.append(dic)

  return res

In [ ]:
cdArea = get_cdAreas_city(28)
print(cdArea)
print('要素数：' + str(len(cdArea)))

[{'cdArea': '28100,28101,28102,28105,28106,28107,28108,28109,28110,28111,28201,28202,28203,28204,28205,28206,28207,28208,28209,28210,28212,28213,28214,28215,28216,28217,28218,28219,28220,28221,28222,28223,28224,28225,28226,28227,28228,28229,28301,28365,28381,28382,28442,28443,28446,28464,28481,28501,28585,28586'}]
要素数：1


## estat-APIのレスポンスをファイルに保存する関数

In [ ]:
def get_estat_response(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

In [ ]:
def conv_estat_response_to_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # TABLE_INF
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

  # VALUE
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  # VALUEをDataFrameに変換
  df_res = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df_res = pd.merge(df_res, df_class, on=key_name, how='outer')

  # 統計情報を追加
  df_res['statsDataId'] = TABLE_INF['@id']
  df_res['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df_res

In [ ]:
def  format_estat_dataframe(df_arg):
  # 必要な列だけ抽出
  df_res =  df_arg[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','@area_name','$','@unit']]

  # 列名の変更
  columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','@area_name':'areaName','$':'value','@unit':'unit'}
  df_res = df_res.rename(columns=columns)

  # 欠損データ削除
  df_res = df_res.dropna()

  # '-'を'0'に置換
  df_res = df_res.replace('-', '0')

  # categoryNameから不要な情報（categoryCode）を削除
  df_res['categoryName'] = df_res.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df_res['timeCode'] = df_res.apply(lambda x: x['timeCode'][:4], 1)

  return df_res

In [ ]:
def format_estat_dataframe_withrank(df_arg):

  # 年度リストを作成
  times = df_arg['timeCode'].tolist()
  times = list(set(times))

  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  # 年度毎に順位を付与
  for time in times:

    # 単年度のdataを抽出
    df_time = df_arg.copy()[df_arg['timeCode'] == time]

    # valueを数値に変換して、'rank'を付与
    df_time = df_time.astype({'value': float})
    df_time['rank'] = df_time.rank(ascending=False)['value'].astype(int).astype(str)

    # 結合
    df_res = pd.concat([df_res, df_time])
    df_res = df_res.astype({'value': str})

  return df_res

In [ ]:
def get_estat_dataframe(params):
  # estat-APIのレスポンス取得
  res = get_estat_response(params)
  # print(res)

  # レスポンスをDataFrameに変換
  df = conv_estat_response_to_dataframe(res)

  # DataFrameを整形
  df = format_estat_dataframe(df)

  # 順位を付与
  df = format_estat_dataframe_withrank(df)

  return df

## estat-APIのレスポンスをBigQueryに保存

In [ ]:
def set_estat_params():
  df_cardmng = get_cards_management_dataframe()
  df_cardlist = get_cardliist(df_cardmng)
  
  res = []
  for index, row in df_cardlist.iterrows():
    # indexの取得
    cardId = index[0]
    governmentType = index[2]
    statsDataId = index[3]

    # 都道府県の場合
    if governmentType == 'prefecture':

      areas = set_estat_params_area(governmentType)

      params = []
      for area in areas:
        area['statsDataId'] = statsDataId if type(statsDataId) == 'int' else str(statsDataId).zfill(10)
        area['cdCat01'] = row['categoryCode']
        params.append(area)

      res.append({
          'path' : 'statistics-hyogo/test/{}.json'.format(cardId),
          'params' : params
        })

    # 市区町村の場合
    if governmentType == 'city':

      pref_codes = [d.get('prefCode') for d in get_resas_preflist()]

      for pref_code in pref_codes:
        areas = set_estat_params_area(governmentType,pref_code)
        
        params = []
        for area in areas:
          area['statsDataId'] = statsDataId if type(statsDataId) == 'int' else str(statsDataId).zfill(10)
          area['cdCat01'] = row['categoryCode']
          params.append(area)

        res.append({
            'path' : 'statistics-hyogo/test/{}_{}.json'.format(cardId,pref_code),
            'params' : params
          })

  return res

In [ ]:
cards = set_estat_params()

print(cards)

'path'と'params'のリストを作成する。
'statsDataId'はdataframeからcsvコピーした際に数値に変換されてしまう。

In [ ]:
for card in cards:

  # 保存先path
  path = card['path']

  df_res = pd.DataFrame(index=[], columns=[])
  
  for params in card['params'] :
    res = get_estat_response(params)

    df = conv_estat_response_to_dataframe(res)

    df_res = pd.concat([df_res, df])

  df = format_estat_dataframe(df_res)
  
  df = format_estat_dataframe_withrank(df)  


  # jsonで保存
  res_json = df.reset_index().to_json(path,orient='records',force_ascii=False)

estat-APIのレスポンスを整形してファイルに保存